In [1]:
#Las Siguientes lineas no requieren de ninguna modificación, 
#son las librerias necesarias para el funcionamiento de la libreta
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
import pandas as pd
output_notebook()


Loading BokehJS ...

In [2]:
#Las siguientes instrucciones definen la función que establece comunicación con un servidor ThingsBoard y 
#por lo tanto no necesitan modificación
import requests
import pandas as pd
import json
import datetime
epoch = datetime.datetime.utcfromtimestamp(0)
def unix_time_millis(dt):
  return (dt - epoch).total_seconds() * 1000
def get_config(file="local.ini", sec="DEFAULT"):
  """Reads a section of a configpareser INI file"""
  import configparser
  config = configparser.ConfigParser()
  config.read(file)
  section = config[sec]
  out = {}
  for i in section.items():
    out[i[0]] = i[1]
  return out
class TB:
  def __init__(self, c):
    self.c = c
    print(c)
  def request(self):
    auth_url = "http://{}:{}/api/auth/login".format(self.c["host"], self.c["port"])
    data = {"username": self.c["tenant"], "password": self.c["password"]}
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    r = requests.post(auth_url, data=json.dumps(data), headers=headers)
    self.jwt_token = json.loads(r.text)['token']
    #print(self.jwt_token)
  #def get_series(self):
    timeseries = "http://{}:{}/api/plugins/telemetry/DEVICE/{}/keys/timeseries".format(self.c["host"], self.c["port"], self.c["device_id"])
    headers = {"Content-Type":"application/json", "X-Authorization": "Bearer {}".format(self.jwt_token)}
    keys = requests.get(timeseries, headers=headers)
#     print(keys.text)
    self.keys = keys.text
#     self.keys = 'ds18b20_T3p1_1'
    #return self.keys
  def labels(self):
    auth_url = "http://{}:{}/api/auth/login".format(self.c["host"], self.c["port"])
    data = {"username": self.c["tenant"], "password": self.c["password"]}
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    r = requests.post(auth_url, data=json.dumps(data), headers=headers)
    self.jwt_token = json.loads(r.text)['token']
    #print(self.jwt_token)
  #def get_series(self):
    timeseries = "http://{}:{}/api/plugins/telemetry/DEVICE/{}/keys/timeseries".format(self.c["host"], self.c["port"], self.c["device_id"])
    headers = {"Content-Type":"application/json", "X-Authorization": "Bearer {}".format(self.jwt_token)}
    keys = requests.get(timeseries, headers=headers)
    print(keys.text)
#     self.keys = keys.text
#     self.keys = 'ds18b20_T3p1_1'
    #return self.keys
  def get_telemetry(self, start_datetime=datetime.datetime(2015,1,1,0,0,0), end_datetime=datetime.datetime.now(), interval=1000, limit=10000000, agg="NONE", key_list=None, drop_cols=None):
    start_m = int(unix_time_millis(start_datetime))
    end_m = int(unix_time_millis(end_datetime))
    if key_list == None:
      key_list = self.keys.replace('"','').replace('[','').replace(']','')
    timeseries = "http://{}:{}/api/plugins/telemetry/DEVICE/{}/values/timeseries?keys={}&startTs={}&endTs={}&interval={}&limit={}&agg={}".format(self.c["host"], self.c["port"], self.c["device_id"], key_list, start_m, end_m, interval, limit, agg)
    headers = {"Content-Type":"application/json", "X-Authorization": "Bearer {}".format(self.jwt_token)}
    telemetry = requests.get(timeseries, headers=headers)
    data_df = pd.read_json(telemetry.text, orient="records")
    if drop_cols != None:
      for col in drop_cols:
        del data_df[col]
    data_tmp = {}
    cnt = 0
    if type(key_list) == str:
      key_list = key_list.split(",")
    for i in key_list:
      data_tmp[i] =  data_df[i].apply(pd.Series)
      data_tmp[i].value = pd.to_numeric(data_tmp[i].value)
      data_tmp[i].rename(columns = {'value':i}, inplace=True)
      if cnt > 0:
        del data_tmp[i]['ts']
      cnt +=1
    frames = []
    for l in key_list:
      frames.append(data_tmp[l])
    data_all = pd.concat(frames, axis=1)
    data_all.ts = pd.to_datetime(data_all.ts, unit='ms')
    data_all.index = data_all['ts']
    del data_all['ts']
#     data_all.index = data_all.index.tz_localize("America/Mexico_City",ambiguous="infer")
    return data_all
  def get_df(self, etiquetas,start_datetime=datetime.datetime(2015,1,1,0,0,0), end_datetime=datetime.datetime.now(), interval=20000, limit=10000000, agg="NONE", key_list=None, drop_cols=None, cache_file=None):
    """Does everything in one step, just a util method"""
    self.request()
#     print(self.keys)
    #self.get_series()
    #out = self.get_telemetry(start_datetime, end_datetime, interval, limit, agg, key_list, drop_cols)
    self.keys = etiquetas
    print(self.keys)
    out = self.get_telemetry(start_datetime, end_datetime, interval, limit, agg, drop_cols)
    self.dataframe = out
    return out
  def save_cache(self, file_path):
    self.dataframe.to_csv(file_path)
  def get_cache(self, file_path):
    self.dataframe = pd.read_csv(file_path)

In [3]:
#Esta linea escanea el archivo "config_gb.ini" el cual debemos editar para
#que los datos de acceso sean los mismos que configuramos en la sonda y poder descargar la información
#almacenada
tb = TB(get_config("config_gbv.ini","GEOTERMIA"))

{'token': 'SzzD371JYP1jeogt7l9u', 'device_id': '99f4cc00-4587-11ea-9ffe-35550336f914', 'tenant': 'tenant@thingsboard.org', 'password': 'OilCeic2Or-', 'host': 'iot.ier.unam.mx', 'port': '8080'}


In [4]:
#Esta instrucción nos permite ver todas las etiquetas de los datos que han sido enviados y
#podemos corroborar que todos nuestros sensores se encuentren en esta lista
tb.labels()

["DesT1_tub_1","DesT1_tub_3","DesT2_tub_2","Hum_tub_1","Hum_tub_2","Hum_tub_3","Pres_tub_1","Pres_tub_2","Pres_tub_3","T1","T1_tub_1","T1_tub_2","T1_tub_3","T2","T2_tub_1","T2_tub_2","T2_tub_3","T3","T4","T5","T6","T7","V_tub_1","V_tub_2","V_tub_3"]


In [5]:
#Estas lineas nos permiten importar todos los datos que se enviaron bajo una misma etiqueta
#se pueden estipular los nombres que se vean mas convenientes para las variables pero se recomienda
#continuar con el formato de ejemplo para no requerir mayor edición mas adelante
A00 = tb.get_df("T1")
A01 = tb.get_df("T2")
A02 = tb.get_df("T3")
A03 = tb.get_df("T4")
A04 = tb.get_df("T5")
A05 = tb.get_df("T6")
A06 = tb.get_df("T7")

T1
T2
T3
T4
T5
T6
T7


In [6]:
A01

,T2
ts,
2020-04-27 21:10:59.491,-127.0
2020-04-27 21:09:57.973,85.0
2020-04-27 21:08:56.394,85.0
2020-04-27 21:07:54.905,85.0
2020-04-27 21:06:53.323,-127.0
...,...
2020-03-20 16:52:57.801,100.0
2020-03-20 16:52:04.212,-127.0
2020-03-20 16:52:02.904,100.0


In [7]:
##a veces estaremos interesados en una ventana especifica de tiempo por lo que estas lineas nos 
#permiten elegir fecha y hora inicial y final de los datos que deseamos guardar, es necesario respetar
# el formato completo para evitar problemas : 'YYYY-MONTH-DD HH:MM:SS' ejemplo: fecha1 = '2020-March-23 16:30:00'
fecha1 = '2020-March-20 16:30:00'
fecha2 = '2020-April-27 12:30:00'
A00 = A00[fecha2:fecha1]
A01 = A01[fecha2:fecha1]
A02 = A02[fecha2:fecha1]
A03 = A03[fecha2:fecha1]
A04 = A04[fecha2:fecha1]
A05 = A05[fecha2:fecha1]
A06 = A06[fecha2:fecha1]

In [11]:
#las siguientes lineas crean una grafica con los datos importados anteriormente y respetando las fechas
#solicitadas,  para cambiar las fechas es necesario volver a correr todas las celdas
import matplotlib.colors as colors

colores = list(colors._colors_full_map.values())

p = figure(title="Temperaturas subsuelo", x_axis_label='time', y_axis_label='Temperature [\xb0 C]',x_axis_type='datetime')
#p.scatter(x=A00.index,y=A00.T1,color=colores[2],legend_label='T1')
#p.scatter(x=A01.index,y=A01.T2,color=colores[4],legend_label='T2')
p.scatter(x=A02.index,y=A02.T3,color=colores[5],legend_label='T3')
p.scatter(x=A03.index,y=A03.T4,color=colores[8],legend_label='T4')
#p.scatter(x=A04.index,y=A04.T5,color=colores[10],legend_label='T5')
#p.scatter(x=A05.index,y=A05.T6,color=colores[20],legend_label='T6')
#p.scatter(x=A06.index,y=A06.T7,color=colores[34],legend_label='T7')
p.legend.location = "top_left"
show(p)


In [9]:
#Finalmente
#Estas lineas nos crearan un archivo CSV en la misma carpeta donde se encuentre este cuarderno 
#Se opto por un CSV por variable para facilitar la exportación de los datos que se necesiten a excel
# se pueden exportar individualmente las variables de interes 
A00.T1.to_csv('T1.csv')
A01.T2.to_csv('T2.csv')
A02.T3.to_csv('T3.csv')
A03.T4.to_csv('T4.csv')
A04.T5.to_csv('T5.csv')
A05.T6.to_csv('T6.csv')
A06.T7.to_csv('T7.csv')